# Topic Modeling
### Without Preprocessing

In [1]:
import json
import os
import random
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
import spacy
# import coreferee
# nlp = spacy.load('en_core_web_trf')
# nlp.add_pipe('coreferee')
nlp = spacy.load("en_core_web_sm")

2022-08-28 17:34:20.594186: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-28 17:34:20.594260: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


#### Define Parameters

In [2]:
n_components = 5 #Topics To generate
n = 20 #Transcripts to use
max_df=20
min_df=3
n_terms = 10

## Load Data

In [3]:
# pos = ["CCONJ","NUM","PRON","PROPN","PUNCT","SCONJ","SYM","X"]
# pos = ["ADJ","ADP","ADV","AUX","CCONJ","DET","INTJ","NOUN","NUM","PART","PRON","PROPN","PUNCT","SCONJ","SYM","VERB","X"]
pos = ["ADJ","AUX","CCONJ","DET","INTJ","NUM","PART","PRON","PROPN","PUNCT","SCONJ","SYM","X"]
def clean_text(text, coref = False, lemma = True, pos_exclusions = pos):
    doc = nlp(text)

    if coref:
        new_text = ""
        for i  in range(len(doc)):
            tokens = doc[i]
            if doc._.coref_chains.resolve(tokens):
                new_text += doc._.coref_chains.resolve(tokens)[0].text
            else:
                new_text += tokens.text
            new_text+= " "
        doc = nlp(new_text)
    text = ""
    for token in doc:
        if token.pos_ not in pos_exclusions:
            text += token.lemma_ if lemma else token.text
            text += " "
    return text

In [4]:
def create_df_multi(n=25):
    list_of_text = []
    dir = 'data/aligned data/c=4'

    files = [filename for filename in os.listdir(dir)]
    sampled_files = random.choices(files,k=n)

    for filename in sampled_files:
        # choice = random.choice(os.listdir(dir))
        # print(choice)
        f = open(dir+'/'+filename)
        data = json.load(f)
        choice = random.choice(os.listdir(dir))
        # print(choice)
        f = open(dir+'/'+choice)
        data = json.load(f)

        for x in data:
            for y in x['TURNS']:
                text = ' '.join(y['UTTERANCES'])

                text = clean_text(text)
                list_of_text.append(text)
    df = pd.DataFrame(list_of_text)
    return df

In [5]:
df = create_df_multi(n)
print(df.shape)
df.head()

(49539, 1)


,0
0,laughter damn welcome to tonight episode of bu...
1,unfortunately down tonight get plague from nee...
2,just come
3,almost have woman at table
4,almost have back back back for while now so re...


## Vectorize Data



- Count Vectorizer
- Tfidf Vectorizer

Use both with each model

In [8]:
# tfidf=TfidfVectorizer(stop_words='english',max_df=.7,min_df=2,token_pattern=r'(?u)\b[A-Za-z]+\b')

tfidf = TfidfVectorizer(stop_words='english', 
    max_df=max_df,
    min_df=min_df,
    token_pattern=r'(?u)\b[A-Za-z]+\b'
    )
tfidf_sparse = tfidf.fit_transform(df[0])
print(tfidf_sparse.shape)
tfidf_df = pd.DataFrame(tfidf_sparse.toarray().transpose(),
                   index=tfidf.get_feature_names_out())
tfidf_df.tail()

(49539, 3283)


,0,1,2,3,4,5,6,7,8,9,...,49529,49530,49531,49532,49533,49534,49535,49536,49537,49538
youtube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ziggurat,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zigzag,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zombie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zone,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
cv = CountVectorizer(stop_words='english', 
    max_df=max_df,
    min_df=min_df,
    token_pattern=r'(?u)\b[A-Za-z]+\b'
    )
cv_sparse = cv.fit_transform(df[0])
print(cv_sparse.shape)
cv_df = pd.DataFrame(cv_sparse.toarray().transpose(),
                   index=cv.get_feature_names_out())
cv_df.tail()

(49539, 3283)


,0,1,2,3,4,5,6,7,8,9,...,49529,49530,49531,49532,49533,49534,49535,49536,49537,49538
youtube,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ziggurat,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zigzag,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zombie,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zone,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Define Output Functions

In [12]:
def print_top_terms(n_components, topics, terms, n_terms = 10):
    for x in range(n_components):
        topic = x
        components = topics[:,topic]
        top_term_indices = components.argsort()[-n_components:]
        top_terms = np.array(terms)[top_term_indices]
        
        print(f'Topic {x}:\t{top_terms.tolist()}')


## PCA

In [3]:
pca = PCA(n_components=n_components)
topics = pca.fit_transform(cv_df)
print_top_terms(n_components=n_components, topics=topics, terms=cv.get_feature_names_out())

NameError: name 'PCA' is not defined

In [20]:
pca = PCA(n_components=n_components)
topics = pca.fit_transform(tfidf_df)
print_top_terms(n_components=n_components, topics=topics, terms=tfidf.get_feature_names_out())

Topic 0:	['risk', 'loud', 'nap', 'pray', 'ow', 'lesson', 'goodness', 'yay', 'math', 'earring']
Topic 1:	['rip', 'goodness', 'disengage', 'darling', 'squeak', 'goddamn', 'hm', 'scare', 'dryad', 'downstair']
Topic 2:	['prisoner', 'warhammer', 'downstairs', 'cannon', 'wink', 'brothel', 'alcohol', 'goddamn', 'fucker', 'math']
Topic 3:	['scare', 'math', 'troll', 'goddamn', 'barrack', 'bust', 'lesson', 'yay', 'alcohol', 'comment']
Topic 4:	['grit', 'cookie', 'oil', 'nap', 'loud', 'disengage', 'scoop', 'yay', 'hm', 'bust']
Topic 5:	['intimidation', 'disengage', 'stealthe', 'brothel', 'fucker', 'goodness', 'math', 'chicken', 'lesson', 'bust']
Topic 6:	['yay', 'bust', 'charm', 'copy', 'dryad', 'squeak', 'nap', 'alcohol', 'hm', 'fucker']
Topic 7:	['slot', 'bastard', 'scoop', 'rip', 'button', 'barrack', 'squeak', 'loud', 'fucker', 'disengage']
Topic 8:	['poof', 'pause', 'stomach', 'hm', 'stealthe', 'dryad', 'loud', 'disengage', 'alcohol', 'chicken']
Topic 9:	['prisoner', 'assist', 'lesson', 'yay'

## LDA

In [17]:
lda = LatentDirichletAllocation(n_components=n_components)
topics = lda.fit_transform(cv_df)
print_top_terms(n_components=n_components, topics=topics, terms=cv.get_feature_names_out())

Topic 0:	['hood', 'dog', 'credit', 'confuse', 'glove', 'goddamn', 'lack', 'gauntlet', 'sake', 'compatriot']
Topic 1:	['foyer', 'scoot', 'stomach', 'halfling', 'chicken', 'fold', 'spray', 'bard', 'forgery', 'troll']
Topic 2:	['crownsguard', 'obelisk', 'patrol', 'dip', 'camera', 'scoop', 'herd', 'loud', 'clap', 'servant']
Topic 3:	['refer', 'foremost', 'stealthe', 'comment', 'study', 'port', 'react', 'bullet', 'flip', 'brazier']
Topic 4:	['rapier', 'downstair', 'engage', 'law', 'ow', 'treat', 'halve', 'demon', 'risk', 'gunshot']
Topic 5:	['halfway', 'endeavor', 'bastard', 'stake', 'fucker', 'outward', 'restore', 'location', 'disengage', 'alcohol']
Topic 6:	['forgive', 'territory', 'goal', 'cliff', 'theme', 'season', 'oil', 'panel', 'sinkhole', 'cleanse']
Topic 7:	['goodness', 'intimidation', 'elbow', 'rip', 'podcast', 'lesson', 'twitter', 'shave', 'cask', 'clothing']
Topic 8:	['hint', 'truly', 'bucket', 'separate', 'downstairs', 'dryad', 'scare', 'earring', 'mace', 'choke']
Topic 9:	['fe

In [18]:
lda = LatentDirichletAllocation(n_components=n_components)
topics = lda.fit_transform(tfidf_df)
print_top_terms(n_components=n_components, topics=topics, terms=tfidf.get_feature_names_out())

Topic 0:	['swinge', 'thumb', 'rotate', 'gain', 'wink', 'flip', 'darling', 'troll', 'copy', 'alcohol']
Topic 1:	['dope', 'guys', 'midnight', 'rag', 'cancel', 'pair', 'gut', 'dip', 'scout', 'downstair']
Topic 2:	['rug', 'pause', 'demon', 'camera', 'react', 'risk', 'halve', 'rip', 'disengage', 'comment']
Topic 3:	['deserve', 'evocation', 'fury', 'swear', 'gap', 'caravan', 'assist', 'stake', 'season', 'loud']
Topic 4:	['knife', 'clap', 'bard', 'butt', 'treat', 'oil', 'yay', 'goddamn', 'lesson', 'math']
Topic 5:	['spray', 'proficiency', 'preparation', 'compatriot', 'meeting', 'divide', 'revenge', 'longsword', 'tickle', 'slot']
Topic 6:	['religion', 'tinker', 'difference', 'foremost', 'parchment', 'enchant', 'brothel', 'intimidation', 'dryad', 'fucker']
Topic 7:	['bullet', 'separate', 'prisoner', 'confuse', 'scoop', 'stealthe', 'barrack', 'choke', 'goodness', 'earring']
Topic 8:	['elbow', 'crownsguard', 'bastard', 'glove', 'cannon', 'cask', 'ow', 'downstairs', 'scare', 'nap']
Topic 9:	['cook

## SVD

In [15]:
svd = TruncatedSVD(n_components=n_components)
topics = svd.fit_transform(cv_df)
print_top_terms(n_components=n_components, topics=topics, terms=cv.get_feature_names_out())

Topic 0:	['forgive', 'jet', 'agreement', 'compound', 'celestial', 'awake', 'sinkhole', 'ankheg', 'cleanse', 'swarm']
Topic 1:	['height', 'clasp', 'lid', 'fold', 'fox', 'veil', 'swell', 'roil', 'mute', 'loss']
Topic 2:	['gala', 'joint', 'snuck', 'stance', 'demeanor', 'deem', 'pitfall', 'detonate', 'bash', 'forgery']
Topic 3:	['mood', 'wandering', 'banner', 'infiltrate', 'demand', 'sever', 'siege', 'colony', 'varn', 'k']
Topic 4:	['religion', 'southward', 'bond', 'amphitheater', 'outward', 'friendship', 'civilization', 'erathis', 'bench', 'dome']
Topic 5:	['introduce', 'replace', 'tal', 'dorei', 'truly', 'knack', 'champion', 'childhood', 'impression', 'headmaster']
Topic 6:	['hail', 'fiction', 'locate', 'explore', 'credit', 'devise', 'stack', 'stumble', 'dragonborn', 'guild']
Topic 7:	['scuffle', 'portrait', 'rocket', 'bat', 'devil', 'rakshasa', 'lattice', 'avert', 'transition', 'prison']
Topic 8:	['safely', 'creation', 'celebration', 'halt', 'decay', 'pilgrimage', 'civilization', 'quest

In [16]:
svd = TruncatedSVD(n_components=n_components)
topics = svd.fit_transform(tfidf_df)
print_top_terms(n_components=n_components, topics=topics, terms=tfidf.get_feature_names_out())

Topic 0:	['glove', 'crownsguard', 'stealthe', 'treat', 'chocolate', 'math', 'milk', 'dryad', 'chicken', 'earring']
Topic 1:	['scare', 'scoop', 'cannon', 'rip', 'chicken', 'disengage', 'goddamn', 'dryad', 'earring', 'downstair']
Topic 2:	['season', 'crownsguard', 'react', 'downstair', 'goodness', 'copy', 'hm', 'comment', 'lesson', 'math']
Topic 3:	['barrack', 'hm', 'darling', 'choke', 'goddamn', 'copy', 'chicken', 'disengage', 'bust', 'comment']
Topic 4:	['thumb', 'hm', 'oil', 'goodness', 'cookie', 'copy', 'alcohol', 'math', 'fucker', 'bust']
Topic 5:	['ow', 'tickle', 'squeak', 'alcohol', 'nap', 'dryad', 'goodness', 'hm', 'bust', 'lesson']
Topic 6:	['butt', 'halve', 'copy', 'disengage', 'dryad', 'nap', 'charm', 'squeak', 'alcohol', 'fucker']
Topic 7:	['glove', 'brothel', 'stealthe', 'charm', 'halve', 'goodness', 'rip', 'hm', 'alcohol', 'disengage']
Topic 8:	['liar', 'intimidation', 'disengage', 'troll', 'charm', 'copy', 'lesson', 'fucker', 'goddamn', 'hm']
Topic 9:	['button', 'plenty', 

## NMF

In [13]:
nmf = NMF(n_components=n_components, max_iter=500)
topics = nmf.fit_transform(cv_df)
print_top_terms(n_components=n_components, topics=topics, terms=cv.get_feature_names_out())

Topic 0:	['forgive', 'jet', 'agreement', 'compound', 'celestial', 'awake', 'ankheg', 'sinkhole', 'cleanse', 'swarm']
Topic 1:	['height', 'clasp', 'lid', 'fold', 'fox', 'veil', 'swell', 'roil', 'mute', 'loss']
Topic 2:	['gala', 'joint', 'snuck', 'stance', 'demeanor', 'deem', 'pitfall', 'detonate', 'bash', 'forgery']
Topic 3:	['mood', 'wandering', 'infiltrate', 'demand', 'banner', 'sever', 'siege', 'varn', 'colony', 'k']
Topic 4:	['religion', 'civilization', 'southward', 'bond', 'amphitheater', 'outward', 'friendship', 'erathis', 'bench', 'dome']
Topic 5:	['introduce', 'replace', 'dorei', 'tal', 'truly', 'knack', 'champion', 'childhood', 'impression', 'headmaster']
Topic 6:	['hail', 'fiction', 'locate', 'explore', 'credit', 'devise', 'stack', 'stumble', 'dragonborn', 'guild']
Topic 7:	['represent', 'portrait', 'rocket', 'bat', 'devil', 'rakshasa', 'lattice', 'avert', 'transition', 'prison']
Topic 8:	['safely', 'celebration', 'creation', 'halt', 'decay', 'pilgrimage', 'quest', 'abominatio

In [14]:
nmf = NMF(n_components=n_components, max_iter=200)
topics = nmf.fit_transform(tfidf_df)
print_top_terms(n_components=n_components, topics=topics, terms=tfidf.get_feature_names_out())

/home/patrick/anaconda3/envs/metis/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/home/patrick/anaconda3/envs/metis/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Topic 0:	['salvation', 'seize', 'statue', 'rally', 'stranger', 'restore', 'chop', 'enchant', 'pray', 'earring']
Topic 1:	['massaging', 'cow', 'bond', 'bum', 'choke', 'chip', 'yay', 'chocolate', 'milk', 'downstair']
Topic 2:	['wreathe', 'wedge', 'deserve', 'timing', 'engage', 'bastard', 'activate', 'wrong', 'warhammer', 'math']
Topic 3:	['shark', 'wipe', 'hover', 'blink', 'poof', 'nap', 'breast', 'drown', 'globule', 'comment']
Topic 4:	['stock', 'debate', 'maker', 'prisoner', 'absolve', 'assassin', 'disengage', 'clanking', 'implication', 'bust']
Topic 5:	['chuck', 'betrayer', 'rob', 'cheat', 'admit', 'circus', 'treat', 'liar', 'thrower', 'lesson']
Topic 6:	['elbow', 'lure', 'fry', 'waffle', 'berry', 'rally', 'egg', 'poof', 'stomach', 'chicken']
Topic 7:	['creativity', 'disengage', 'haversack', 'foremost', 'clink', 'mead', 'mix', 'season', 'goodness', 'alcohol']
Topic 8:	['remnant', 'mug', 'skeleton', 'rating', 'respond', 'wheel', 'rescue', 'fucker', 'dryad', 'squeak']
Topic 9:	['origin'